# Predictive Analysis of Public Transportation Delays

This notebook cleans the provided dirty dataset, engineers features, trains 2 regression models, evaluates them, and exports deliverables to `outputs/`.

In [ ]:
from pathlib import Path

import pandas as pd

from src.transport_delay.pipeline import run_pipeline


In [ ]:
project_dir = Path().resolve().parent
input_csv = project_dir / 'dirty_transport_dataset.csv'
output_dir = project_dir / 'outputs'

out = run_pipeline(input_csv=input_csv, output_dir=output_dir)
out.model_metrics

In [ ]:
out.feature_importance.head(15)

In [ ]:
pd.read_csv(output_dir / 'cleaned_transport_dataset.csv').head()